In [4]:
import torch
from torch_geometric.nn import SplineConv, voxel_grid, max_pool, max_pool_x
import torch.nn.functional as F
import torch_geometric.transforms as T
class nvs_no_skip(torch.nn.Module):
    def __init__(self):
        super(nvs_no_skip, self).__init__()
        self.conv1 = SplineConv(1, 64, dim=3, kernel_size=3)
        self.bn1 = torch.nn.BatchNorm1d(64)
        self.conv2 = SplineConv(64, 128, dim=3, kernel_size=3)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.conv3 = SplineConv(128, 256, dim=3, kernel_size=3)
        self.bn3 = torch.nn.BatchNorm1d(256)
        self.conv4 = SplineConv(256, 512, dim=3, kernel_size=3)
        self.bn4 = torch.nn.BatchNorm1d(512)
        self.fc1 = torch.nn.Linear(64 * 512, 1024)
        self.fc2 = torch.nn.Linear(1024, 2)

    def forward(self, data):
        data.x = F.elu(self.conv1(data.x, data.edge_index, data.edge_attr))
        data.x = self.bn1(data.x)
        cluster = voxel_grid(data.pos, batch=data.batch, size=0.05)
        data = max_pool(cluster, data, transform=T.Cartesian(cat=False))

        data.x = F.elu(self.conv2(data.x, data.edge_index, data.edge_attr))
        data.x = self.bn2(data.x)
        cluster = voxel_grid(data.pos, batch=data.batch, size=0.1)
        data = max_pool(cluster, data, transform=T.Cartesian(cat=False))
        
        data.x = F.elu(self.conv3(data.x, data.edge_index, data.edge_attr))
        data.x = self.bn3(data.x)
        cluster = voxel_grid(data.pos,batch= data.batch, size=0.15)
        data = max_pool(cluster, data, transform=T.Cartesian(cat=False))

        data.x = F.elu(self.conv4(data.x, data.edge_index, data.edge_attr))
        data.x = self.bn4(data.x)
        cluster = voxel_grid(data.pos, batch=data.batch, size=0.25)
        x,_ = max_pool_x(cluster, data.x, batch=data.batch, size=64)

        x = x.view(-1, self.fc1.weight.size(1))
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return x


In [1]:
from torch_geometric.loader import DataLoader
from torch_geometric import seed_everything
from torch_geometric.transforms import Distance, Cartesian
from imports.TrainModel import TrainModel
seed_everything(0)

model
!rm ../data/contact_extraction10/{train,test,val}/processed/*

tm = TrainModel('/home/hussain/tactile/data/contact_extraction1/', model, n_epochs=150, transform=Cartesian(cat=False), features='pol', augment=False)

/bin/bash: /home/hussain/miniconda3/envs/tactile/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [3]:
tm.train_data.get(10)

Data(x=[1877, 2], edge_index=[2, 60064], y=[1, 2], pos=[1877, 2], edge_attr=[60064, 2])

In [2]:
tm.train()

training:   0%|          | 0/150 [00:00<?, ?epoch/s]

  0%|          | 0/386 [00:00<?, ?batch/s]

/home/hussain/tactile/src/imports/TrainModel.py:72: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/home/hussain/miniconda3/envs/tactile/lib/python3.8/site-packages/torch_spline_conv/basis.py", line 10, in spline_basis
                 is_open_spline: torch.Tensor,
                 degree: int) -> Tuple[torch.Tensor, torch.Tensor]:
    return torch.ops.torch_spline_conv.spline_basis(pseudo, kernel_size,
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
                                                    is_open_spline, degree)
RuntimeError: pseudo.size(1) == kernel_size.numel() INTERNAL ASSERT FAILED at "csrc/cuda/basis_cuda.cu":104, please report a bug to PyTorch. Input mismatch


In [7]:
min(tm.val_losses)

0.022077545086204103

In [4]:
tm.test()

0.011305167100377001

In [24]:
import time 
from tqdm.auto import tqdm
t = []
def validate(self):
    loss = 0
    for i, data in enumerate(tqdm(self.val_loader)):
        now = time.time()
        data = data.to('cuda')
        end_point = self.model(data)
        end = time.time() - now
        t.append(end)
        print(end)
        loss += self.loss_func(end_point[0], data.y).detach().item()
    loss /= len(self.val_data)
    return loss

validate(tm)

  0%|          | 0/33 [00:00<?, ?it/s]

0.14057564735412598
0.17262601852416992
0.1955254077911377
0.15732598304748535
0.12617993354797363
0.13708806037902832
0.1367342472076416
0.19402194023132324
0.1806349754333496
0.16005754470825195
0.13409996032714844
0.1878058910369873
0.19119000434875488
0.19316840171813965
0.1704239845275879
0.1675717830657959
0.20548200607299805
0.18075847625732422
0.14330005645751953
0.1307077407836914
0.17769408226013184
0.20035505294799805
0.18609166145324707
0.1927506923675537
0.1511704921722412
0.14408493041992188
0.11974000930786133
0.10378074645996094
0.12209033966064453
0.17664790153503418
0.1758272647857666
0.13147425651550293
0.2036294937133789


0.0227090794536652

In [26]:
import numpy as np
np.mean(t)

0.1633519692854448

In [23]:
tm.model.to('cuda')

nvs_no_skip(
  (conv1): SplineConv(1, 64, dim=3)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): SplineConv(64, 128, dim=3)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): SplineConv(128, 256, dim=3)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): SplineConv(256, 512, dim=3)
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=32768, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=2, bias=True)
)

In [11]:
from imports.ExtractContactCases import ExtractContactCases

ex = ExtractContactCases('/home/hussain/tactile/data/contact_extraction10/', '/home/hussain/tactile/dataset_ENVTACT_new2.bag')
ex.extract()

parsing bag: 0it [00:00, ?it/s]

  0%|          | 0/645 [00:00<?, ?it/s]

saving
386 129 129
